<a href="https://colab.research.google.com/github/RAM-1166/Zeotap_assignment/blob/main/Peteti_Ram_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Load and Inspect Data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Load datasets
customers = pd.read_csv("/content/drive/MyDrive/Zeotap/Customers.csv")
products = pd.read_csv("/content/drive/MyDrive/Zeotap/Products.csv")
transactions = pd.read_csv("/content/drive/MyDrive/Zeotap/Transactions.csv")

# Merge datasets
merged_data = transactions.merge(products, on='ProductID', how='left').merge(customers, on='CustomerID', how='left')

# Rename duplicate columns for clarity
merged_data.rename(columns={"Price_x": "TransactionPrice", "Price_y": "ProductPrice"}, inplace=True)

# Verify merged_data columns
print("Columns in merged_data:", merged_data.columns)

# 1. Overall Customer Activity Analysis
merged_data['SignupDate'] = pd.to_datetime(merged_data['SignupDate'])
merged_data['TransactionDate'] = pd.to_datetime(merged_data['TransactionDate'])

# Calculate customer signup duration before their first transaction
signup_transaction = merged_data.groupby('CustomerID').agg(
    SignupDate=('SignupDate', 'min'),
    FirstTransactionDate=('TransactionDate', 'min'),
    TotalTransactions=('TransactionID', 'count'),
    TotalSpent=('TotalValue', 'sum')
)
signup_transaction['DaysToFirstTransaction'] = (
    signup_transaction['FirstTransactionDate'] - signup_transaction['SignupDate']
).dt.days

# 2. Regional Insights
regional_insights = merged_data.groupby('Region').agg(
    TotalSales=('Quantity', 'sum'),
    TotalRevenue=('TotalValue', 'sum'),
    UniqueCustomers=('CustomerID', 'nunique')
).sort_values(by='TotalRevenue', ascending=False)

# 3. Customer Insights
customer_insights = merged_data.groupby('CustomerID').agg(
    TotalTransactions=('TransactionID', 'count'),
    TotalSpent=('TotalValue', 'sum'),
    AvgTransactionValue=('TotalValue', 'mean'),
    UniqueProductsPurchased=('ProductID', 'nunique'),
    Region=('Region', 'first')
).sort_values(by='TotalSpent', ascending=False)

# 4. Transaction Insights
transaction_insights = merged_data.agg(
    TotalTransactions=('TransactionID', 'count'),
    TotalRevenue=('TotalValue', 'sum'),
    AvgTransactionValue=('TotalValue', 'mean'),
    TotalQuantity=('Quantity', 'sum'),
    UniqueCustomers=('CustomerID', 'nunique')
)

# 5. Product Insights
product_insights = merged_data.groupby('ProductName').agg(
    TotalSales=('Quantity', 'sum'),
    TotalRevenue=('TotalValue', 'sum'),
    AvgPrice=('ProductPrice', 'mean'),  # Use renamed column
    UniqueBuyers=('CustomerID', 'nunique')
).sort_values(by='TotalRevenue', ascending=False)

# First Product Purchased Analysis
first_product_purchases = merged_data.sort_values('TransactionDate').drop_duplicates('CustomerID')
first_product_analysis = first_product_purchases['ProductName'].value_counts()

# Display Results
print("\n--- Customer Activity Analysis ---")
print(signup_transaction.head())

print("\n--- Regional Insights ---")
print(regional_insights)

print("\n--- Customer Insights ---")
print(customer_insights.head())

print("\n--- Transaction Insights ---")
print(transaction_insights)

print("\n--- Product Insights ---")
print(product_insights.head())

print("\n--- First Product Purchased Analysis ---")
print(first_product_analysis)



Columns in merged_data: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'TransactionPrice', 'ProductName', 'Category',
       'ProductPrice', 'CustomerName', 'Region', 'SignupDate'],
      dtype='object')

--- Customer Activity Analysis ---
           SignupDate FirstTransactionDate  TotalTransactions  TotalSpent  \
CustomerID                                                                  
C0001      2022-07-10  2024-01-19 03:12:55                  5     3354.52   
C0002      2022-02-13  2024-02-28 07:44:21                  4     1862.74   
C0003      2024-03-07  2024-02-18 02:50:37                  4     2725.38   
C0004      2022-10-09  2024-02-28 10:16:35                  8     5354.88   
C0005      2022-08-15  2024-03-15 04:08:59                  3     2034.24   

            DaysToFirstTransaction  
CustomerID                          
C0001                          558  
C0002                          745  
C0003           